# 데이터 로드 및 형태 확인

In [101]:
import pandas as pd

In [102]:
kj_new = pd.read_csv('C:/Users/B0113/Desktop/집닥 프로젝트/data/전처리완료/집닥_견적DB_최종_배분P.csv')

In [103]:
kj_new.head()

,견적번호,견적일,유입경로,유입경로_상세,지역,주소_시군구,주소_읍면동,주소_번지,아파트명,전체주소,...,배분P_2,배분P_3,배분P_4,배분P_5,배분P_6,배분P_7,배분P_8,배분P_9,배분P_10,수요_배분P전체기준
0,200037,2019-07-31 21:56,웹,모바일웹,경상남도,창원시 성산구,반림동,8,럭키반림아파트,경상남도 창원시 성산구 반림동 8,...,비율디자인(수락),리안디자인(수락),NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
1,200030,2019-07-31 20:31,앱,아이폰,서울특별시,강동구,명일동,54,한양아파트,서울특별시 강동구 명일동 54,...,디자인스튜디오산들(수락),인테리온(수락),NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
2,199988,2019-07-31 16:26,앱,안드로이드,서울특별시,강서구,가양동,1461,가양2단지아파트,서울특별시 강서구 가양동 1461,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,199970,2019-07-31 15:30,웹,PC웹,서울특별시,성북구,돈암동,644,돈암코오롱하늘채,서울특별시 성북구 돈암동 644,...,리홈디자인갤러리(수락),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
4,199937,2019-07-31 14:31,외부,믹스앤픽스,서울특별시,강동구,암사동,509,선사현대아파트,서울특별시 강동구 암사동 509,...,디자인스튜디오산들(수락),행복공간디자인(수락),NaN,NaN,NaN,NaN,NaN,NaN,NaN,3


# '주소_시군구' 컬럼에서 '주소_시군' 컬럼 만들기
ex. 창원시 성산구 -> 창원시, 강서구 -> 강서구
## split 함수 생성

In [104]:
def split_sgg(row):
    return row.split(' ')[0]

split_sgg('창원시 성산구')

'창원시'

## apply로 새로운 컬럼 생성

In [105]:
kj_new['주소_시군'] = kj_new['주소_시군구'].apply(split_sgg)

# 지역별, 시군구별 groupby

In [111]:
needs = kj_new.groupby(['지역', '주소_시군'])['수요_배분P전체기준'].sum()
needs

지역    주소_시군
강원도   강릉시       49
      고성군        2
      동해시       25
      삼척시        9
      속초시       26
              ... 
충청북도  제천시       28
      증평군        9
      진천군        6
      청주시      168
      충주시       42
Name: 수요_배분P전체기준, Length: 207, dtype: int64

In [123]:
# needs = needs.reset_index()
needs

,지역,주소_시군,수요_배분P전체기준
0,강원도,강릉시,49
1,강원도,고성군,2
2,강원도,동해시,25
3,강원도,삼척시,9
4,강원도,속초시,26
...,...,...,...
202,충청북도,제천시,28
203,충청북도,증평군,9
204,충청북도,진천군,6
205,충청북도,청주시,168


# 시군별 수치자료와 merge

## merge

In [114]:
data = pd.read_excel('../analysis/시군별 수치자료_노후주택포함.xlsx')

In [115]:
data.head()

,지역,주소_시군,인구수_2%표본,이사가구수_2%표본,견적건수,계약건수,수요_배분P전체기준,총세대수_합계,노후주택수,노후주택수15-29년,파트너스수_주소,파트너스수_시공지역,파트너스수_1순위시공지역,성비,입주년도_중앙값
0,강원도,강릉시,2484,128,29,5,NaN,48443,111,97,6,39,10,93.457944,1993.0
1,강원도,고성군,214,15,1,1,NaN,1501,5,2,1,39,4,125.263158,2002.5
2,강원도,동해시,1508,62,13,0,NaN,23607,62,54,0,39,0,96.354167,1995.0
3,강원도,삼척시,878,65,5,1,NaN,13834,62,60,0,39,0,103.712297,1994.0
4,강원도,속초시,1533,109,11,0,NaN,24884,47,39,3,39,0,90.671642,1995.0


In [116]:
data.shape

(229, 15)

In [117]:
new_data = pd.merge(data, needs, how='left', left_on=['지역','주소_시군'], right_on=['지역', '주소_시군'])

In [118]:
new_data.shape

(229, 16)

In [119]:
new_data.head()

,지역,주소_시군,인구수_2%표본,이사가구수_2%표본,견적건수,계약건수,수요_배분P전체기준_x,총세대수_합계,노후주택수,노후주택수15-29년,파트너스수_주소,파트너스수_시공지역,파트너스수_1순위시공지역,성비,입주년도_중앙값,수요_배분P전체기준_y
0,강원도,강릉시,2484,128,29,5,NaN,48443,111,97,6,39,10,93.457944,1993.0,49.0
1,강원도,고성군,214,15,1,1,NaN,1501,5,2,1,39,4,125.263158,2002.5,2.0
2,강원도,동해시,1508,62,13,0,NaN,23607,62,54,0,39,0,96.354167,1995.0,25.0
3,강원도,삼척시,878,65,5,1,NaN,13834,62,60,0,39,0,103.712297,1994.0,9.0
4,강원도,속초시,1533,109,11,0,NaN,24884,47,39,3,39,0,90.671642,1995.0,26.0


In [120]:
new_data.isnull().sum()

지역                 0
주소_시군              0
인구수_2%표본           0
이사가구수_2%표본         0
견적건수               0
계약건수               0
수요_배분P전체기준_x     229
총세대수_합계            0
노후주택수              0
노후주택수15-29년        0
파트너스수_주소           0
파트너스수_시공지역         0
파트너스수_1순위시공지역      0
성비                 1
입주년도_중앙값           1
수요_배분P전체기준_y      30
dtype: int64

## merge 잘 되었는지 확인

data는 229건, needs는 207건 (차이: 22건) 이었는데 new_data에서 30건의 null값 나온 이유? (8건의 데이터는 어디에서?)
- needs 데이터에서 지역별 '-' 행 총 7개
    - 사실상 229건 & 200건 한 것
- 인천광역시 남구는 명칭 변경으로 미추홀구

In [124]:
needs[needs['지역']=='광주광역시']

,지역,주소_시군,수요_배분P전체기준
76,광주광역시,-,0
77,광주광역시,광산구,142
78,광주광역시,남구,81
79,광주광역시,동구,37
80,광주광역시,북구,140
81,광주광역시,서구,123


In [125]:
sg1 = data['주소_시군'].unique()
sg1

array(['강릉시', '고성군', '동해시', '삼척시', '속초시', '양구군', '양양군', '영월군', '원주시',
       '인제군', '정선군', '철원군', '춘천시', '태백시', '평창군', '홍천군', '화천군', '횡성군',
       '가평군', '고양시', '과천시', '광명시', '광주시', '구리시', '군포시', '김포시', '남양주시',
       '동두천시', '부천시', '성남시', '수원시', '시흥시', '안산시', '안성시', '안양시', '양주시',
       '양평군', '여주시', '연천군', '오산시', '용인시', '의왕시', '의정부시', '이천시', '파주시',
       '평택시', '포천시', '하남시', '화성시', '거제시', '거창군', '김해시', '남해군', '밀양시',
       '사천시', '산청군', '양산시', '의령군', '진주시', '창녕군', '창원시', '통영시', '하동군',
       '함안군', '함양군', '합천군', '경산시', '경주시', '고령군', '구미시', '군위군', '김천시',
       '문경시', '봉화군', '상주시', '성주군', '안동시', '영덕군', '영양군', '영주시', '영천시',
       '예천군', '울릉군', '울진군', '의성군', '청도군', '청송군', '칠곡군', '포항시', '광산구',
       '남구', '동구', '북구', '서구', '달서구', '달성군', '수성구', '중구', '대덕구', '유성구',
       '강서구', '금정구', '기장군', '동래구', '부산진구', '사상구', '사하구', '수영구', '연제구',
       '영도구', '해운대구', '강남구', '강동구', '강북구', '관악구', '광진구', '구로구', '금천구',
       '노원구', '도봉구', '동대문구', '동작구', '마포구', '서대문구', '서초구', '성동구', '성북구',
       '송파구

In [126]:
sg2 = needs['주소_시군'].unique()
sg2

array(['강릉시', '고성군', '동해시', '삼척시', '속초시', '원주시', '인제군', '정선군', '춘천시',
       '태백시', '평창군', '홍천군', '횡성군', '가평군', '고양시', '과천시', '광명시', '광주시',
       '구리시', '군포시', '김포시', '남양주시', '동두천시', '부천시', '성남시', '수원시', '시흥시',
       '안산시', '안성시', '안양시', '양주시', '양평군', '여주시', '연천군', '오산시', '용인시',
       '의왕시', '의정부시', '이천시', '파주시', '평택시', '포천시', '하남시', '화성시', '거제시',
       '거창군', '김해시', '밀양시', '사천시', '양산시', '의령군', '진주시', '창녕군', '창원시',
       '통영시', '하동군', '함안군', '함양군', '-', '경산시', '경주시', '구미시', '김천시', '문경시',
       '봉화군', '상주시', '성주군', '안동시', '영주시', '영천시', '울진군', '청도군', '청송군',
       '칠곡군', '포항시', '광산구', '남구', '동구', '북구', '서구', '달서구', '달성군', '수성구',
       '중구', '대덕구', '유성구', '강서구', '금정구', '기장군', '동래구', '부산진구', '사상구',
       '사하구', '수영구', '연제구', '영도구', '해운대구', '강남구', '강동구', '강북구', '관악구',
       '광진구', '구로구', '금천구', '노원구', '도봉구', '동대문구', '동작구', '마포구', '서대문구',
       '서초구', '성동구', '성북구', '송파구', '양천구', '영등포구', '용산구', '은평구', '종로구',
       '중랑구', '세종특별자치시', '울주군', '강화군', '계양구', '남동구', '미추홀구', '부평구', '연수구',


In [127]:
for i in range(len(sg2)):
    if sg2[i] in sg1:
        print('있음')
    else:
        print(sg2[i]+'없음')

있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
-없음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음


In [128]:
needs[needs['지역']=='세종특별자치시']

,지역,주소_시군,수요_배분P전체기준
137,세종특별자치시,세종특별자치시,34


In [129]:
data[data['지역']=='세종특별자치시']

,지역,주소_시군,인구수_2%표본,이사가구수_2%표본,견적건수,계약건수,수요_배분P전체기준,총세대수_합계,노후주택수,노후주택수15-29년,파트너스수_주소,파트너스수_시공지역,파트너스수_1순위시공지역,성비,입주년도_중앙값
149,세종특별자치시,세종특별자치시,29458,1913,14,1,NaN,104670,34,27,9,47,0,94.763636,2015.0


## 필요 없는 컬럼 제거, 컬럼명 변경

In [130]:
new_data = new_data.drop('수요_배분P전체기준_x', axis=1)
new_data.shape

(229, 15)

In [132]:
new_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 229 entries, 0 to 228
Data columns (total 15 columns):
지역               229 non-null object
주소_시군            229 non-null object
인구수_2%표본         229 non-null int64
이사가구수_2%표본       229 non-null int64
견적건수             229 non-null int64
계약건수             229 non-null int64
총세대수_합계          229 non-null int64
노후주택수            229 non-null int64
노후주택수15-29년      229 non-null int64
파트너스수_주소         229 non-null int64
파트너스수_시공지역       229 non-null int64
파트너스수_1순위시공지역    229 non-null int64
성비               228 non-null float64
입주년도_중앙값         228 non-null float64
수요_배분P전체기준_y     199 non-null float64
dtypes: float64(3), int64(10), object(2)
memory usage: 28.6+ KB


In [133]:
new_data = new_data.rename(columns={'수요_배분P전체기준_y':'수요_배분P전체기준'})

In [134]:
new_data.columns

Index(['지역', '주소_시군', '인구수_2%표본', '이사가구수_2%표본', '견적건수', '계약건수', '총세대수_합계',
       '노후주택수', '노후주택수15-29년', '파트너스수_주소', '파트너스수_시공지역', '파트너스수_1순위시공지역', '성비',
       '입주년도_중앙값', '수요_배분P전체기준'],
      dtype='object')

## 결측치 처리

In [138]:
new_data.loc[new_data['수요_배분P전체기준'].isnull(), '수요_배분P전체기준'] = 0

In [139]:
new_data[new_data['수요_배분P전체기준'].isnull()]

,지역,주소_시군,인구수_2%표본,이사가구수_2%표본,견적건수,계약건수,총세대수_합계,노후주택수,노후주택수15-29년,파트너스수_주소,파트너스수_시공지역,파트너스수_1순위시공지역,성비,입주년도_중앙값,수요_배분P전체기준


In [141]:
new_data.isnull().sum()

지역               0
주소_시군            0
인구수_2%표본         0
이사가구수_2%표본       0
견적건수             0
계약건수             0
총세대수_합계          0
노후주택수            0
노후주택수15-29년      0
파트너스수_주소         0
파트너스수_시공지역       0
파트너스수_1순위시공지역    0
성비               1
입주년도_중앙값         1
수요_배분P전체기준       0
dtype: int64

In [145]:
new_data.to_excel('../analysis/시군별 수치자료_노후주택포함.xlsx', header=True, index=False, encoding='UTF-8')